In [1]:
import sys
sys.path.append('/home/silja/fatepredict/fatepredict_experiments')

In [2]:
import zarr
import numpy as np
import networkx as nx
from skimage.measure import regionprops
#from segmentation import segment_stats
from linajea_cost_test import get_merge_graph_from_array
from linajea_cost_test import get_merge_graph
from funlib.math import decode64, encode64
import networkx as nx
from candidate_graph import iterate_tree

In [3]:
z = zarr.open('anno_alice_T2030_tiny.zarr','r')
gt_image = z['gt_trackimage'][:]
raw = z['Raw'][:]
fragments = z['Fragments'][:]


In [4]:
# annotation label is 10000, 10001, 10004, 10005, 10007, 10010, 10013, 10016, 10017 pick them out
# annotaion is made by merge frgments by napari tool
gt_mask = gt_image >=10000
gt_image = gt_mask * gt_image

In [5]:
# create annotation graph by gt_trackimage

def create_annotation_graph(gt_image):
    '''
    gt_image: ndarray image by (t,z,y,x) each cell has an annotated label. for exmaple 10000, 10001, .... 

    return
    anno_graph: nx.Digrph() which create a graph with edges.
                these edges connected node are generated by encode cell.
                e.g label n will be encode to u at time 0
                                    encode to v at time 1
                then the edge will be (v, u, source = v, target = u)
    
    '''
    time = gt_image.shape[0]
    anno_graph = nx.DiGraph()
    for label in np.unique(gt_image):
        if label == 0:
            continue   
        for t in range(time):
            mask = gt_image[t]==label
            regions = regionprops(mask.astype(int))
            for props in regions:
                z0, y0, x0 = props.centroid
                u = encode64((t, int(z0), int(y0), int(x0), int(props.area)),bits=[9,12,12,12,19])
            mask = gt_image[t]==label
            regions = regionprops(mask.astype(int))
            for props in regions:
                z0, y0, x0 = props.centroid
                v = encode64((t, int(z0), int(y0), int(x0), int(props.area)),bits=[9,12,12,12,19])
            anno_graph.add_edge(v, u, source = v, target = u)
    return anno_graph


In [6]:
# get the id list of one time frame
from segmentation import segment_stats
id, positions, _ = segment_stats(gt_image[0],0)

t = 0
merge_tree = z['Merge_tree/Merge/'+str(t)]
scores = z['Merge_tree/Scoring/'+str(t)]
merge_tree = get_merge_graph_from_array(merge_tree,scores)

# count the number of how many annotation id in merge tree at time t
def count_anno_id(id,merge_tree):
    '''
    merge_tree: nx.Digrph merge_tree create by waterz 
    id: ndarray (n,) 1d array store all ids in GT image
    correct_id_num: int count the number of how many annotation node(id) in merge tree
    '''
    mt_id = list(merge_tree.nodes)
    correct_id_num = len(set(id)&set(mt_id))
    return correct_id_num 

correct_id_num = count_anno_id(id,merge_tree)


# get the sovler output

In [7]:
# read .gexf file which created by run_candidate_graph.ipynb
graph = nx.read_graphml('test.graphml')

'''
NOTE:
the Digraph nodes will convert to string after nx.read_graphml
the can didate_graph.node should be 
{19140315621363712: {'t': 0, 'z': 8, 'y': 12, 'x': 2, 'score': 0.04924517869949341, 'parent': 53339542484161024, 'id': 19140315621363712}
the graph will convert 19140315621363712 to '19140315621363712'
{'19140315621363712': {'t': 0, 'z': 8, 'y': 12, 'x': 2, 'score': 0.04924517869949341, 'parent': 53339542484161024, 'id': 19140315621363712}
'''

"\nNOTE:\nthe Digraph nodes will convert to string after nx.read_graphml\nthe can didate_graph.node should be \n{19140315621363712: {'t': 0, 'z': 8, 'y': 12, 'x': 2, 'score': 0.04924517869949341, 'parent': 53339542484161024, 'id': 19140315621363712}\nthe graph will convert 19140315621363712 to '19140315621363712'\n{'19140315621363712': {'t': 0, 'z': 8, 'y': 12, 'x': 2, 'score': 0.04924517869949341, 'parent': 53339542484161024, 'id': 19140315621363712}\n"

In [8]:
def get_select_graph(graph,key='selected'):
    '''
    get a sub-graph form graph

    graph: nx.Digraph() candidate graph with some edges has attribute 'select' = True

    return
    select_graph: nx.Digraph()  sub-graph only have edges from candidate graph with 'select' is True
  
    '''
    select_graph = nx.DiGraph()
    for u,v,data in graph.edges(data=True):
        if data[key]:
            select_graph.add_edge(v, u, source = v, target = u)
            select_graph.add_node(u, **graph.nodes[u])
            select_graph.add_node(v, **graph.nodes[v])
    return select_graph

select_graph = get_select_graph(graph)


def pick_select_edge(graph):
    '''
    graph: nx.Digraph() candidate graph with edge has attribute 'select' = True

    return
    edges: list of all edges in graph
    select_edges: list of edges with attribute 'select' == True
 
    '''
    edges = []
    selected_edges = []
    for u,v,data in graph.edges(data=True):
        edges.append((u,v))
        if data['selected']:
            selected_edges.append((u,v))
    return edges, selected_edges




def find_devison_node(select_graph):
    ''' 
    find the devision node
    select_graph: nx.Digraph()  sub-graph only have edges from graph with 'select' is True

    return
    devision_nodes: list of nodes wihch are split
    '''
    devision_nodes = []
    for node in select_graph.nodes:
        if select_graph.out_degree(node) == 2:
            devision_nodes.append(node)
    return devision_nodes
    

In [9]:
track_data = []
# creat a list for tracking node
track_id_node = []
track_id = 0
for node in select_graph.nodes:
    cor = decode64(int(node),dims=5,bits=[9,12,12,12,19])
    track_data.append([track_id,cor[0],cor[1],cor[2],cor[3]])
    track_id_node.append(node)
    track_id += 1

In [10]:
def select_nodes_t(select_graph,t):
    '''
    pick nodes at time t in select_graph

    select_graph: nx.Digraph()
    t: int 

    return
    nodes_t: a string list wich contains the nodes at time t 

    '''
    nodes_t = []
    for node in select_graph.nodes:
        cor = decode64(int(node),dims = 5,bits=[9,12,12,12,19])
        if cor[0]== t:
            nodes_t.append(node)
    return nodes_t
# pick out the start node

node_start = select_nodes_t(select_graph,0)
print('the ids of nodes in t0 are:', node_start)

the ids of nodes in t0 are: ['1020578718816256', '1794634908964352', '8584995398486528', '11892360728284160', '950218593931776', '13616583976948736', '387045272852992', '14425618334619136', '1125934287555072', '1337143618181632', '1020389761225728', '12420306700342784', '1724248995398144', '1337238096969728', '11751795078464512', '1126054584385536', '3131469249647616', '1372233480018432', '4433239475228160', '5770314342471680', '70600682897408', '9429446111005184', '27795748470920192', '352084251639808', '2005715371687936', '3870495678139392', '3307477059766272', '25860668156548096', '6333384552877056', '809352231393792', '950184223705600', '34234522901939712', '5453646393247232', '28323531230021120', '6825836952946176', '34199252666160128', '4538973003186688', '1161256111177728', '39582607616248320', '3061229369167872', '844545189216256', '10555337411133952', '2286992805068800', '1759450543162880', '7142659481143296', '8303520449037824', '1442567904297984', '809309334145024', '4222150

In [11]:
def find_path(graph, start, end):
    '''
    find a path between star and end in graph

    graph: nx.Digrph
    start: string  the start point 
    end: string  the end point

    '''
    queue = [(start, [start])]
    while queue:
        (node, path) = queue.pop(0)
        for neighbor in graph.successors(node):
            if neighbor == end:
                yield path + [end]
            else:
                queue.append((neighbor, path + [neighbor]))
                
def find_all_paths(graph, start):
    '''
    find all path with a start point

    graph: nx.Digrph
    start: string  the start point  
    '''
    paths = []
    for node in graph.nodes():
        paths.extend(list(find_path(graph, start, node)))
    return [path for i, path in enumerate(paths) if not any(path == subpath[:len(path)] for subpath in paths[:i] + paths[i+1:])]

def find_repeated_nodes(paths):
    node_counts = {}
    for path in paths:
        for node in path:
            node_counts[node] = node_counts.get(node, 0) + 1
    repeated_nodes = set([node for node, count in node_counts.items() if count > 1])
    return repeated_nodes

def pick_nodes_once(lst,select_graph):
    '''

    lst : a list of paths of one node (from find_all_path)
        e.g node 1 has paths [[1,2],[1,2,3]]
    select_graph : nx.Digraph
    
    '''
    output = []
    nodes = set()
    for item in lst:
        new_item = []
        for node in item:
            if node not in nodes:
                nodes.add(node)
                new_item.append(node)
                if select_graph.out_degree(node) == 2:
                    output.append(new_item)
                    new_item = []                     
        if new_item:
            output.append(new_item)
    return output

In [12]:
# create a labels list that prevent napari error  (napari can only read labels under int32)
labels = []
for node in select_graph.nodes:
    labels.append(node)

In [14]:
track_data = []

l=0
num_node = 0
cell_mask = np.zeros(fragments.shape)
track_graph = {}
#track_graph = nx.DiGraph()# graph for arboretum
for start in node_start:    
    paths = find_all_paths(select_graph, start)
    pick = pick_nodes_once(paths,select_graph)

    for path in pick:
        label = labels.index(path[0]) # choose first node id as label
        for node in path:
            cor = decode64(int(node),dims=5,bits=[9,12,12,12,19])
            track_data.append([label,cor[0],cor[1],cor[2],cor[3]])
            if select_graph.out_degree(node) == 2:
                for neighbor in select_graph.out_edges(node):
                        track_graph[labels.index(neighbor[1])] = [label]
            t = cor[0]
            
            ids = z['Fragment_stats/id/'+str(t)]
            merge_tree = z['Merge_tree/Merge/'+str(t)]
            scores = z['Merge_tree/Scoring/'+str(t)]
            merge_tree = get_merge_graph_from_array(merge_tree,scores)

            if merge_tree.has_node(int(node)):
                sub_tree = iterate_tree(merge_tree,int(node))
                for a in sub_tree:
                    if a in list(ids):
                        # a is the leave node id
                        # get the label of this leave node
                        label_a = list(ids).index(a)+1
                        cell_mask[t][fragments[t] == label_a] = label

        

    if len(paths)>1:
        l=l+1
    #print('******************')
        print('the paths are',paths)
        #print('******************')
        for path in pick:
            print('sub-set of paths:',path)
        print('******************')



the paths are [['1337143618181632', '1161213163607553', '13335040276567554'], ['1337143618181632', '1513048298756097', '668606188754946']]
sub-set of paths: ['1337143618181632']
sub-set of paths: ['1161213163607553', '13335040276567554']
sub-set of paths: ['1513048298756097', '668606188754946']
******************
the paths are [['34234522901939712', '211192140268033'], ['34234522901939712', '5629628389524481', '2005663836275714']]
sub-set of paths: ['34234522901939712']
sub-set of paths: ['211192140268033']
sub-set of paths: ['5629628389524481', '2005663836275714']
******************
the paths are [['5453646393247232', '1759287323919361', '598168685248514'], ['5453646393247232', '1900050586272769', '422298364412418']]
sub-set of paths: ['5453646393247232']
sub-set of paths: ['1759287323919361', '598168685248514']
sub-set of paths: ['1900050586272769', '422298364412418']
******************
the paths are [['844545189216256', '351963979973633', '3307459825371650'], ['844545189216256', '45

In [15]:
import napari
viewer = napari.Viewer()
viewer.add_image(raw[3])
viewer.add_labels(gt_image)
viewer.add_labels(fragments)
tracks = napari.layers.Tracks(track_data,   
                              graph=track_graph, 
                              name="track_test")
viewer.add_labels(cell_mask.astype(int))
viewer.add_layer(tracks)
viewer.window.add_plugin_dock_widget(plugin_name="napari-arboretum", 
                                     widget_name="Arboretum")



(<napari._qt.widgets.qt_viewer_dock_widget.QtViewerDockWidget at 0x7f6bafca92d0>,
 <napari_arboretum.plugin.Arboretum at 0x7f6c27240160>)

In [37]:
# example for track

'''
               /  5  
           /  4 - 6
      / 2 / - 3 - 7
    1/- 8 - - 9 - 10

'''

g = nx.DiGraph()
g.add_edge(2,1)
g.add_edge(8,1)

g.add_edge(3,2)
g.add_edge(4,2)

g.add_edge(5,4)
g.add_edge(6,4)

g.add_edge(7,3)

g.add_edge(9,8)
g.add_edge(10,9)

node_start=(1)
lst = [[1,2,4,5],[1,2,4,6],[1,2,3,7],[1,8,9,10]]
#lst = [['1337143618181632', '1161213163607553', '13335040276567554'], ['1337143618181632', '1513048298756097', '668606188754946']]
track_graph = {}
output = []
nodes = set()
for item in lst:
    new_item = []
    for node in item:
        if node not in nodes:
            nodes.add(node)
            new_item.append(node)
            if g.in_degree(node) == 2:
                for neighbor in g.in_edges(node):
                    track_graph[neighbor[0]] = neighbor[1]
                output.append(new_item)
                new_item = []
                      
    if new_item:
        output.append(new_item)

for i in output:
    print(i)
print(track_graph)

[1]
[2]
[4]
[5]
[6]
[3, 7]
[8, 9, 10]
{2: 1, 8: 1, 3: 2, 4: 2, 5: 4, 6: 4}
